In [32]:
import pandas as pd
import regex as re

In [33]:
# data = pd.read_csv('Q3_dataset.txt')
data = pd.read_csv('Q3_dataset.txt', sep='\t', header=None)

In [34]:
# take only the rows that contain 'qid:4' use regular expression
data = data[data[0].str.contains('qid:4 ')]

In [35]:
# store data of concern in  a new text file
data.to_csv('Q3_dataset_new.txt', sep='\t', header=None, index=None)

In [ ]:
# 